# Shallow & Deep Copies

## Descrição

Em diversas linguagens de programação, há o conceito de ponteiros de memória. Nestes casos, podemos pensar que cada ponteiro seria o equivalente a uma variável alocada na memória.

Um único ponteiro pode estar associado a mais de uma variável independente (relação de 1:N). Nestes casos, a alteração em uma variável (a) resultará na mesma alteração na variável (b).

## Tipos de cópia e manipulação dos dados

Em situações envolvendo a gestão destes ponteiros, por exemplo, em casos de otimização de gestão de memória, surge a necessidade da gestão da cópia de dados, e, portanto de seus respectivos ponteiros.

Assim, origina-se o conceito de Cópia raza (shallow copy) e cópia profunda (deep copy).

### Shallow Copy: cópia raza

Este tipo de copia é tido por padrão (default) no Python.

Diversas classes (e sub-classes) se utilizam do *shallow copy* para respectiva cópia de dados: list, dict, class.

Embora seja o padrão da linguagem, é possível alterarmos este comportamento de copia de dados em nossas rotinas.

In [10]:
from typing import List

In [14]:
# Exemplo envolvendo uma lista
def criarListaDeLegumes() -> List[str]:
  minhaListaDeLegumes = list()

  minhaListaDeLegumes.append("batatas")
  minhaListaDeLegumes.append("mandioca")
  minhaListaDeLegumes.append("mandioquinha")
  minhaListaDeLegumes.append("cenoura")

  return minhaListaDeLegumes

def imprimir_elementos_da_lista(lista:list) -> None:
  for i in lista:
    print(i)


In [15]:

minhaListaDeLegumes = criarListaDeLegumes()

imprimir_elementos_da_lista(minhaListaDeLegumes)

batatas
mandioca
mandioquinha
cenoura


In [5]:
minhaListaDeSupermercado = minhaListaDeLegumes

minhaListaDeSupermercado.append("Laranja")
minhaListaDeSupermercado.append("Maracuja")
minhaListaDeSupermercado.append("Abacaxi")

In [9]:


elementoASerTestado = "Laranja"

if (elementoASerTestado in minhaListaDeLegumes):
  raise AttributeError(f"Minha lista de legumes não pode conter {elementoASerTestado}")

else:
  imprimir_elementos_da_lista(minhaListaDeLegumes)

AttributeError: ignored

Notar que o exemplo acima resultou na cópia das informações da **suposta** lista de supermercado à lista de legumes.

Para evitarmos este tipo de alteração da lista original (a.k.a., retroagindo a lista primária), devemos nos utilizar de uma cópia profunda dos dados; ou seja, agora, cada elemento da lista inicial terá um novo espaço de memória para seu armazenamento.

In [16]:
## Executando a mesma rotina acima, mas agora com deepy copy
minhaListaDeLegumes = criarListaDeLegumes()
minhaListaDeSupermercado = minhaListaDeLegumes.copy()

minhaListaDeSupermercado.append("Laranja")
minhaListaDeSupermercado.append("Maracuja")
minhaListaDeSupermercado.append("Abacaxi")


if (elementoASerTestado in minhaListaDeLegumes):
  raise AttributeError(f"Minha lista de legumes não pode conter {elementoASerTestado}")

else:
  imprimir_elementos_da_lista(minhaListaDeLegumes)

batatas
mandioca
mandioquinha
cenoura


Abaixo, outro excerto é apresentado, extraído [daqui](https://docs.python.org/3/library/stdtypes.html#list)

Neste exemplo, os elementos da lista são listas vazias. Ao contrário do que se esperaria, cada um destes itens fazem referência a um único ponteiro de memória (relação 3:1); assim, ao se alterar um elemento de uma lista filha, a mesma alteração ocorre nas demais listas irmãs.


In [23]:
# criando uma lista (parental) que contém 3 outras listas (filhas)
lists = [[]] * 3
print(lists)

# alterando apenas a 1° lista filha, por meio de uma inserção do elemento 3.
lists[0].append(3)
print("\n Comportamento:")
print("\t", lists)


[[], [], []]

 Comportamento:
	 [[3], [3], [3]]


Para solucionar o problema, temos 2 soluções:

1) Criar corretamente nossa lista parental de forma a promover múltiplicidade de ponteiros, instanciação de apenas 1 ponteiro para cada lista filha contida na lista parental

2) Forçar uma Deep-Copy da lista parental, antes de realizarmos uma alteração dos elementos ali contidos.

In [24]:
# Exemplo 1) Instanciando corretamente nossa lista de 2 dimensões:

lists = [[] for i in range(3)]
lists[0].append(3)
lists[1].append(5)
lists[2].append(7)
print("\n Comportamento:")
print("\t", lists)


 Comportamento:
	 [[3], [5], [7]]


In [27]:
# criando uma lista (parental) que contém 3 outras listas (filhas)
lists = [[]] * 3
print(lists)

# Forçando um Deep-Copy dos elementos da lista parental:
import copy

lists = [copy.deepcopy(i) for i in lists]

# alterando apenas a 1° lista filha, por meio de uma inserção do elemento 3.
lists[0].append([3])
print("\n Comportamento:")
print("\t", lists)

[[], [], []]

 Comportamento:
	 [[[3]], [], []]


## Sobre-escrita do método padrão de cópia

Abaixo é apresentado um exemplo de implementação de uma classe customizada de tal forma que toda e qualquer cópia realizada nela retorne sempre uma nova instância de si mesma, resultando em uma cópia profunda de si mesma.

In [49]:
class MinhaClasse:
  ClassCounter = 0
  def __init__(self, id=None):
    if (id is None):
      MinhaClasse.ClassCounter += 1
      self.Id = MinhaClasse.ClassCounter
    else:
      self.Id = id

  def __str__(self):
    return f"{self.Id}"

  def copy(self):
      return self.__copy__()

  def __copy__(self):
    return self.__deepcopy__()

  def __deepcopy__(self):
      return MinhaClasse(self.Id)

  def __eq__(self, other:"MinhaClasse") -> bool:
    if (not isinstance(other, MinhaClasse)):
      return False

    else:
      condition = all([
                       (self.Id == other.Id),
                       (id(self) != id(other) )

                       ])

      return condition


instancia = MinhaClasse()

instanciaCopiada = instancia.copy()

# Teste de igualdade
if (instanciaCopiada == instancia):
  print("Instâncias equivalentes, mas não iguais, pois estão em lugares diferentes da memória do computador")
  print("Id exclusivo da instância original: ", id(instancia))
  print("Id exclusivo da instância copiada:  ", id(instanciaCopiada))
else:
  print("a instância copiada não é idêntica à parental")

# Teste de identidade
if (instanciaCopiada is instancia):
  print("Instâncias iguais")
  print(instancia, instanciaCopiada)

else:
  print("a instância copiada é diferente da parental")


Instâncias equivalentes, mas não iguais, pois estão em lugares diferentes da memória do computador
Id exclusivo da instância original:  134403481690288
Id exclusivo da instância copiada:   134403481696768


Referências:

*   [Documentação Python](https://docs.python.org/3/library/copy.html)
*   List item

